In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import cv2
from tqdm import tqdm
import random
from stardist import _draw_polygons
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import skimage
from scipy.optimize import dual_annealing, minimize
from tifffile import imsave, imread
import trackpy as tp

from tracking_utils import *

ModuleNotFoundError: No module named 'tracking_utils'

In [ ]:
#model_name = 'stardist_trained'          # stardist model trained for 50 epochs on simulated synthetic dataset
model_name = 'modified_2D_versatile_fluo' # stardist model trained for 150 epochs on simulated dataset starting from the pretrained 2D versatile fluo model
model = StarDist2D(None, name = model_name, basedir = './models/')
video_selection =  "25b25r-1"#"49b1r"

if video_selection == "25b25r-1":
    xmin, ymin, xmax, ymax = 95, 30, 535, 470 
    merge_present = False   
elif video_selection == "25b25r-2":
    xmin, ymin, xmax, ymax = 82, 13, 410, 342 
    merge_present = False   
elif video_selection == "49b1r":
    xmin, ymin, xmax, ymax = 20, 50, 900, 930
    merge_present = True
    merge_frame = 32269


save_path       = f'./{video_selection}/{model_name}/'
source_path     = f'./data/{video_selection}.mp4'
system_name     = f'{video_selection} system'
nDrops = 50

video = cv2.VideoCapture(source_path)
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video has {n_frames} frames with a resolution of {w}x{h} and a framerate of {fps} fps")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 5))
ax.imshow(get_frame(video, n_frames -1, xmin, ymin, xmax, ymax, w, h, True))
ax1.imshow(get_frame(video, n_frames -1, xmin, ymin, xmax, ymax, w, h, False))
plt.show()

In [ ]:
test_verb = False
detect_verb = False
link_verb = False
interp_verb = False

startFrame = 0
endFrame = n_frames - 1

In [ ]:
if test_verb: 
    n_samples = 1000
    test_detection(n_samples, n_frames, nDrops, video_selection, model, model_name, video, xmin, ymin, xmax, ymax, w, h, save_path)

In [ ]:
if detect_verb:
    print(f'Processing from {int(startFrame/fps)} s to {int(endFrame/fps)} s')
    sample_frames = np.arange(startFrame, endFrame, 1, dtype=int)
    raw_detection_df = detect_features(sample_frames, False, video_selection, model, model_name, video, xmin, ymin, xmax, ymax, w, h, save_path)
    
    n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values
    fig, ax = plt.subplots(2, 2, figsize = (8, 4))
    ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
    ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
    ax[0, 1].plot(raw_detection_df.r, '.')
    ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
    ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
    ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
    ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
    ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
    plt.tight_layout()
    plt.savefig(save_path + f'raw_features_{model_name}_{startFrame}_{endFrame}.png', dpi = 500)
    plt.close()
else:
    raw_detection_df = pd.read_parquet(save_path + f'raw_detection_{video_selection}_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')
    sample_frames = raw_detection_df.frame.unique()

    n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values
    fig, ax = plt.subplots(2, 2, figsize = (8, 4))
    ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
    ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
    ax[0, 1].plot(raw_detection_df.r, '.')
    ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
    ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
    ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
    ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
    ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
    plt.tight_layout()
    plt.savefig(save_path + f'raw_features_{model_name}_{startFrame}_{endFrame}.png', dpi = 500)
    plt.close()

In [ ]:
if merge_present:
    err_frames_pre_merge = np.where(raw_detection_df.loc[raw_detection_df.frame < merge_frame].groupby('frame').count().x.values != nDrops)[0]
    err_frames_post_merge = merge_frame + np.where(raw_detection_df.loc[raw_detection_df.frame >= merge_frame].groupby('frame').count().x.values != nDrops-1)[0] 
    print(f'Number of errors: {len(err_frames_pre_merge)} / {len(sample_frames[:merge_frame])} --> {len(err_frames_pre_merge)/len(sample_frames[:merge_frame])*100:.2f}%')
    print(f'Number of errors: {len(err_frames_post_merge)} / {len(sample_frames[merge_frame:])} --> {len(err_frames_post_merge)/len(sample_frames[merge_frame:])*100:.2f}%')
    condition = np.ediff1d(err_frames_pre_merge)
    condition[condition == 1] = True
    condition[condition != 1] = False
    if 1 in condition:
        max_n_of_consecutive_errs_pre_merge = max(np.diff(np.where(np.concatenate(([condition[0]], condition[:-1] != condition[1:], [True])))[0])[::2])
        print(f'Max number of consecutive errors pre merge: {max_n_of_consecutive_errs_pre_merge}')
    else:
        max_n_of_consecutive_errs_pre_merge = 0
        print(f'Max number of consecutive errors pre merge: 0')
    condition = np.ediff1d(err_frames_post_merge)
    condition[condition == 1] = True
    condition[condition != 1] = False
    if 1 in condition:
        max_n_of_consecutive_errs_post_merge = max(np.diff(np.where(np.concatenate(([condition[0]], condition[:-1] != condition[1:], [True])))[0])[::2])
        print(f'Max number of consecutive errors pre merge: {max_n_of_consecutive_errs_post_merge}')
    else:
        max_n_of_consecutive_errs_post_merge = 0
        print(f'Max number of consecutive errors pre merge: 0')
else:
    err_frames = np.where(raw_detection_df.groupby('frame').count().x.values != nDrops)[0]
    print(f'Number of errors: {len(err_frames)} / {len(sample_frames)} --> {len(err_frames)/len(sample_frames)*100:.2f}%')
    condition = np.ediff1d(err_frames)
    condition[condition == 1] = True
    condition[condition != 1] = False
    if 1 in condition:
        max_n_of_consecutive_errs = max(np.diff(np.where(np.concatenate(([condition[0]], condition[:-1] != condition[1:], [True])))[0])[::2])
        print(f'Max number of consecutive errors: {max_n_of_consecutive_errs}')
    else:
        max_n_of_consecutive_errs = 0
        print(f'Max number of consecutive errors: 0')

if merge_present: 
    traj_test = raw_detection_df.loc[raw_detection_df.frame.isin(np.append(err_frames_pre_merge, err_frames_post_merge))]
else: 
    traj_test = raw_detection_df.loc[raw_detection_df.frame.isin(err_frames)]
fig, ax = fig, ax = plt.subplots(1, 1, figsize=(10, 10))
def update_graph(frame):
    df = traj_test.loc[(traj_test.frame == frame), ["x", "y", "r"]]
    for i in range(len(df)):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = df.r.values[i]
    graph2.set_data(get_frame(video, frame, xmin, ymin, xmax, ymax, w, h, False))
    title.set_text(f'{system_name} Tracking -- t = {round(frame/fps, 1)} s')
    return graph

title = ax.set_title(f'{system_name} Tracking -- t = {0} s')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = traj_test.loc[(traj_test.frame == min(traj_test.frame.unique())), ["x", "y", "r"]]
graph = []
for i in range(len(df)):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), df.r.values[i], color = 'red',\
                                           fill = False, alpha = 0.5, linewidth=1)))
graph2 = ax.imshow(get_frame(video, 0, xmin, ymin, xmax, ymax, w, h, False))
ani = matplotlib.animation.FuncAnimation(fig, update_graph, traj_test.frame.unique(), interval = 5, blit=False)
writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
ani.save(f'./{save_path}/tracking_video_errors.mp4', writer=writer, dpi = 200)
plt.close()

In [ ]:
if link_verb:
    if merge_present:
        print('Linking trajectories...')
        cutoff = 100

        ## PRE MERGE
        t = tp.link_df(raw_detection_df.loc[raw_detection_df.frame < merge_frame], cutoff,\
                                 memory = max_n_of_consecutive_errs_pre_merge, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
        t = t.sort_values(['frame', 'particle'])
        trajectories_pre_merge = tp.filter_stubs(t, 25)
        # CREATE COLOR COLUMN AND SAVE DF
        n = max(trajectories_pre_merge.particle)
        print(f'N of droplets pre merge: {n + 1}')
        random.seed(5)
        colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
        for i in range(max(trajectories_pre_merge.particle)+1-n):
            colors.append('#00FFFF')
        c = []
        for p in t.particle:
            c.append(colors[p])
        trajectories_pre_merge['color'] = c
        trajectories_pre_merge = trajectories_pre_merge.reset_index(drop=True)
        trajectories_pre_merge.to_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_pre_merge.parquet', index = False)

        ## POST MERGE
        t = tp.link_df(raw_detection_df.loc[raw_detection_df.frame >= merge_frame], cutoff,\
                                  memory = max_n_of_consecutive_errs_post_merge, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
        t = t.sort_values(['frame', 'particle'])
        trajectories_post_merge = tp.filter_stubs(t, 25)
        # CREATE COLOR COLUMN AND SAVE DF
        n = max(trajectories_post_merge.particle)
        print(f'N of droplets post merge: {n + 1}')
        random.seed(5)
        colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
        for i in range(max(trajectories_post_merge.particle)+1-n):
            colors.append('#00FFFF')
        c = []
        for p in t.particle:
            c.append(colors[p])
        trajectories_post_merge['color'] = c
        trajectories_post_merge = trajectories_post_merge.reset_index(drop=True)
        trajectories_post_merge.to_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_post_merge.parquet', index = False)
    else:
        print('Linking trajectories...')
        cutoff = 100
        t = tp.link_df(raw_detection_df, cutoff, memory = max_n_of_consecutive_errs, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
        #print(t)
        t = t.sort_values(['frame', 'particle'])
        trajectories = tp.filter_stubs(t, 25)
        # CREATE COLOR COLUMN AND SAVE DF
        n = max(trajectories.particle)
        print(f'N of droplets: {n + 1}')
        random.seed(5)
        colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
        for i in range(max(trajectories.particle)+1-n):
            colors.append('#00FFFF')
        c = []
        for p in t.particle:
            c.append(colors[p])
        trajectories['color'] = c
        trajectories = trajectories.reset_index(drop=True)
        trajectories.to_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet', index = False)
else:
    if merge_present:
        print('Importing linked trajectories...')
        trajectories_pre_merge = pd.read_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_pre_merge.parquet')
        trajectories_post_merge = pd.read_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_post_merge.parquet')
    else:
        print('Importing linked trajectories...')
        trajectories = pd.read_parquet(save_path + f'raw_tracking_{video_selection}_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')

In [ ]:
if interp_verb:
    if merge_present:
        print('Interpolating trajectories...')
        tqdm.pandas()
        interp_trajectories_pre_merge = trajectories_pre_merge.groupby('particle').progress_apply(interpolate_trajectory)
        interp_trajectories_pre_merge = interp_trajectories_pre_merge.reset_index(drop=True)
        interp_trajectories_pre_merge['particle'] = interp_trajectories_pre_merge['particle'].astype(int)
        interp_trajectories_pre_merge = interp_trajectories_pre_merge.sort_values(['frame', 'particle'])
        interp_trajectories_pre_merge.to_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_pre_merge.parquet', index=False)

        interp_trajectories_post_merge = trajectories_post_merge.groupby('particle').apply(interpolate_trajectory)
        interp_trajectories_post_merge = interp_trajectories_post_merge.reset_index(drop=True)
        interp_trajectories_post_merge['particle'] = interp_trajectories_post_merge['particle'].astype(int)
        interp_trajectories_post_merge = interp_trajectories_post_merge.sort_values(['frame', 'particle'])
        interp_trajectories_post_merge.to_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_post_merge.parquet', index=False)
    else:
        print('Interpolating trajectories...')
        tqdm.pandas()
        interp_trajectories = trajectories.groupby('particle').progress_apply(interpolate_trajectory)
        interp_trajectories = interp_trajectories.reset_index(drop=True)
        interp_trajectories['particle'] = interp_trajectories['particle'].astype(int)
        interp_trajectories = interp_trajectories.sort_values(['frame', 'particle'])
        interp_trajectories.to_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet', index=False)
else:
    if merge_present:
        print('Importing interpolated trajectories...')
        interp_trajectories_pre_merge = pd.read_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_pre_merge.parquet')
        interp_trajectories_post_merge = pd.read_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_post_merge.parquet')
    else:
        print('Importing interpolated trajectories...')
        interp_trajectories = pd.read_parquet(save_path + f'interpolated_tracking_{video_selection}_modified_2D_versatile_fluo_{startFrame}_{endFrame}.parquet')

In [ ]:
if 1:
    if merge_present:
        interp_trajectories_subsampled_pre_merge = interp_trajectories_pre_merge.loc[interp_trajectories_pre_merge.frame.isin(interp_trajectories_pre_merge.frame.unique()[::int(fps/10)]), ["x", "y", "r", "frame", "particle", "color"]]
        interp_trajectories_subsampled_pre_merge['frame'] = np.array(interp_trajectories_subsampled_pre_merge.frame/int(fps/10)).astype(int)
        smooth_trajs_pre_merge = get_smooth_trajs(interp_trajectories_subsampled_pre_merge, windLen = 10, orderofPoly = 4)
        ts1 = smooth_trajs_pre_merge.copy()
        ts1["color"] = ts1["color"].astype("category")
        ts1["particle"] = pd.to_numeric(ts1["particle"], downcast="integer")
        ts1["frame"] = pd.to_numeric(ts1["frame"], downcast="integer")
        ts1["x"] = pd.to_numeric(ts1["x"], downcast="float")
        ts1["y"] = pd.to_numeric(ts1["y"], downcast="float")
        ts1["r"] = pd.to_numeric(ts1["r"], downcast="float")
        ts1 = ts1.sort_values(['frame', 'particle'])
        display(ts1.memory_usage(deep=True))
        display(ts1.dtypes)
        reduction = ts1.memory_usage(deep=True).sum() / smooth_trajs_pre_merge.memory_usage(deep=True).sum()
        print(f" Reduced to {reduction*100:0.4f} % of the original size")
        ts1.to_parquet(save_path + f'{video_selection}_subsampled{int(fps/10)}_smoothed_windlen10_orderofpoly4_pre_merge', index=False, engine='pyarrow', partition_cols=['particle'])

        interp_trajectories_subsampled_post_merge = interp_trajectories_post_merge.loc[interp_trajectories_post_merge.frame.isin(interp_trajectories_post_merge.frame.unique()[::int(fps/10)]), ["x", "y", "r", "frame", "particle", "color"]]
        interp_trajectories_subsampled_post_merge['frame'] = np.array(interp_trajectories_subsampled_post_merge.frame/int(fps/10)).astype(int)
        smooth_trajs_post_merge = get_smooth_trajs(interp_trajectories_subsampled_post_merge, windLen = 10, orderofPoly = 4)
        ts2 = smooth_trajs_post_merge.copy()
        ts2["color"] = ts2["color"].astype("category")
        ts2["particle"] = pd.to_numeric(ts2["particle"], downcast="integer")
        ts2["frame"] = pd.to_numeric(ts2["frame"], downcast="integer")
        ts2["x"] = pd.to_numeric(ts2["x"], downcast="float")
        ts2["y"] = pd.to_numeric(ts2["y"], downcast="float")
        ts2["r"] = pd.to_numeric(ts2["r"], downcast="float")
        ts2 = ts2.sort_values(['frame', 'particle'])
        display(ts2.memory_usage(deep=True))
        display(ts2.dtypes)
        reduction = ts2.memory_usage(deep=True).sum() / smooth_trajs_post_merge.memory_usage(deep=True).sum()
        print(f" Reduced to {reduction*100:0.4f} % of the original size")
        ts2.to_parquet(save_path + f'{video_selection}_subsampled{int(fps/10)}_smoothed_windlen10_orderofpoly4_post_merge', index=False, engine='pyarrow', partition_cols=['particle'])
    else:
        interp_trajectories_subsampled = interp_trajectories.loc[interp_trajectories.frame.isin(interp_trajectories.frame.unique()[::int(fps/10)]), ["x", "y", "r", "frame", "particle", "color"]]
        interp_trajectories_subsampled['frame'] = np.array(interp_trajectories_subsampled.frame/int(fps/10)).astype(int)
        smooth_trajs = get_smooth_trajs(interp_trajectories_subsampled, windLen = 10, orderofPoly = 4)
        ts2 = smooth_trajs.copy()
        ts2["color"] = ts2["color"].astype("category")
        ts2["particle"] = pd.to_numeric(ts2["particle"], downcast="integer")
        ts2["frame"] = pd.to_numeric(ts2["frame"], downcast="integer")
        ts2["x"] = pd.to_numeric(ts2["x"], downcast="float")
        ts2["y"] = pd.to_numeric(ts2["y"], downcast="float")
        ts2["r"] = pd.to_numeric(ts2["r"], downcast="float")
        ts2 = ts2.sort_values(['frame', 'particle'])
        display(ts2.memory_usage(deep=True))
        display(ts2.dtypes)
        reduction = ts2.memory_usage(deep=True).sum() / smooth_trajs.memory_usage(deep=True).sum()
        print(f" Reduced to {reduction*100:0.4f} % of the original size")
        ts2.to_parquet(save_path + f'{video_selection}_subsampled{int(fps/10)}_smoothed_windlen10_orderofpoly4', index=False, engine='pyarrow', partition_cols=['particle'])

In [ ]:
n_feature_per_frame = interp_trajectories.groupby('frame').count().x.values
fig, ax = plt.subplots(2, 2, figsize = (8, 4))
ax[0, 0].plot(interp_trajectories.frame.unique(), n_feature_per_frame, '.')
ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
ax[0, 1].plot(interp_trajectories.r, '.')
ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
ax[1, 0].scatter(interp_trajectories.r, interp_trajectories.eccentricity, s=0.1)
ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
ax[1, 1].scatter(interp_trajectories.r, interp_trajectories.prob, s=0.1)
ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
plt.tight_layout()
plt.savefig(save_path + f'interp_features_{model_name}_{startFrame}_{endFrame}.png', dpi = 500)
plt.close()

In [ ]:
fig, ax = fig, ax = plt.subplots(1, 1, figsize=(10, 10))
def update_graph(frame):
    df = interp_trajectories.loc[(interp_trajectories.frame == frame), ["x", "y", "r", 'color']]
    for i in range(len(df)):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = df.r.values[i]
    graph2.set_data(get_frame(video, frame, xmin, ymin, xmax, ymax, w, h, False))
    title.set_text(f'{system_name} Tracking -- t = {round(frame/fps, 1)} s')
    return graph

title = ax.set_title(f'{system_name} Tracking -- t = {0} s')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = interp_trajectories.loc[(interp_trajectories.frame == min(interp_trajectories.frame.unique())), ["x", "y", "r", 'color']]
graph = []
for i in range(len(df)):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), df.r.values[i], color = df.color.values[i],\
                                           fill = False)))
graph2 = ax.imshow(get_frame(video, 0, xmin, ymin, xmax, ymax, w, h, False))
ani = matplotlib.animation.FuncAnimation(fig, update_graph, interp_trajectories.frame.unique(), interval = 5, blit=False)
writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
ani.save(f'./{save_path}/tracking_video2.mp4', writer=writer, dpi = 200)
plt.close()

In [ ]:
test_frames = [12336,  12337,  12338,  12339,  12340,  12341, 12343]
missing_droplet_id = 43
df1 = trajectories.loc[(trajectories.frame < max(test_frames) + 15) & (trajectories.frame > min(test_frames) - 15) & (trajectories.particle == 43)]
df2 = interp_trajectories.loc[(interp_trajectories.frame < max(test_frames) + 15) & (interp_trajectories.frame > min(test_frames) - 15) & (interp_trajectories.particle == 43)]
df3 = smooth_trajs.loc[(smooth_trajs.frame < max(test_frames) + 15) & (smooth_trajs.frame > min(test_frames) - 15) & (smooth_trajs.particle == 43)]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 5))
ax[0,0].plot(df1.x, df1.y, 'o', label = 'Raw', zorder = 20)
ax[0,0].plot(df2.x, df2.y, '--o', label = 'Interpolated', zorder = 10)
ax[0,0].set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Position interpolation')
ax[0,0].legend()
ax[1,0].plot(df1.frame, df1.r, 'o', label = 'Raw', zorder = 20)
ax[1,0].plot(df2.frame, df2.r, '--o', label = 'Interpolated', zorder = 10)
ax[1,0].set(xlabel = 'Frame', ylabel = 'Radius [px]', title = 'Radius interpolation')
ax[1,0].legend()
ax[0,1].plot(df1.frame, df1.prob, 'o', label = 'Raw', zorder = 20)
ax[0,1].plot(df2.frame, df2.prob, '--o', label = 'Interpolated', zorder = 10)
ax[0,1].set(xlabel = 'Frame', ylabel = 'Probability', title = 'Probability interpolation')
ax[0,1].legend()
ax[1,1].plot(df1.frame, df1.area, 'o', label = 'Raw', zorder = 20)
ax[1,1].plot(df2.frame, df2.area, '--o', label = 'Interpolated', zorder = 10)
ax[1,1].set(xlabel = 'Frame', ylabel = 'Area [px]', title = 'Area interpolation')
ax[1,1].legend()
plt.tight_layout()
#plt.savefig(save_path + f'interp_example_linear.png', dpi = 500)
plt.show()

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(12, 5))
ax.plot(df1.x, df1.y, 'o', label = 'Original', zorder = 20)
ax.plot(df2.x, df2.y, '--o', label = 'Interpolated', zorder = 10)
ax1.plot(df3.x, df3.y, '--o', label = 'Smoothed', zorder = 10)
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Raw trajectory')
ax1.set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Smoothed trajectory')
ax.legend()
plt.savefig(save_path + f'interp_smoothing_example_linear.png', dpi = 500)
plt.show()

In [ ]:
_NORMALIZATION_CONSTANT = 2.2522836210436354

def _mollifier(t):
    """Standard mollifier corresponding to epsilon = 1."""
    return _NORMALIZATION_CONSTANT*np.exp(1.0/(t*t - 1.0)) if abs(t) < 1.0 else 0.0

def _mollify(u, k):
    """Returns the discrete mollification of the given signal with window size `k`."""
    if not isinstance(k, int):
        raise TypeError(f"Window size must be an integer: {k=}")
    if k < 1:
        raise ValueError(f"Window size must be at least 1: {k=}")
    if len(u.shape) > 1:
        raise ValueError(f"Input signal must be one-dimensional: {u}")
    n = u.size
    Mu = np.copy(u)
    for j in range(k, n - k):
        Mu[j] = sum(_mollifier(i/k)*u[j - i] for i in range(1 - k, k))/k
    return Mu

def mollify(u, k):
    """Mollification filter with corrected tails."""
    n = u.size
    padded = np.zeros(n + 2*k)
    padded[:k] = u[0]
    padded[k:k + n] = u
    padded[k + n:] = u[-1]
    mollified = _mollify(padded, k)
    return mollified[k:n + k]

def get_mollified_trajs(trajs, order):
    trajs_ret = trajs.copy()
    trajs_ret['x'] = trajs_ret.groupby('particle')['x'].transform(lambda x: mollify(x, order))
    trajs_ret['y'] = trajs_ret.groupby('particle')['y'].transform(lambda y: mollify(y, order))
    return trajs_ret

In [ ]:
test_x = mollify(interp_trajectories.loc[interp_trajectories.particle == 43].x.values, 2)
test_y = mollify(interp_trajectories.loc[interp_trajectories.particle == 43].y.values, 2)

In [ ]:
fig, (ax, ax1, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharex=True, sharey=True)
ax.plot(df1.x, df1.y, 'o', label = 'Original', zorder = 20)
ax.plot(df2.x, df2.y, '--o', label = 'Interpolated', zorder = 10)
ax1.plot(df3.x, df3.y, '--o', label = 'Smoothed', zorder = 10)
ax3.plot(test_x[df1.frame.min():df1.frame.max()], test_y[df1.frame.min():df1.frame.max()], '--o', label = 'Mollified', zorder = 10)
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Raw trajectory')
ax1.set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Smoothed trajectory')
ax.legend()
#plt.savefig(save_path + f'interp_smoothing_example_linear.png', dpi = 500)
plt.show()

# BEST SMOOTHING ?

In [ ]:
interp_trajectories_initial_part = interp_trajectories.loc[interp_trajectories.frame < 10000]
smooth_trajs = get_smooth_trajs(interp_trajectories_initial_part, windLen = 17, orderofPoly = 10)
df1 = interp_trajectories_initial_part.loc[(interp_trajectories_initial_part.particle == 43)]
df2 = smooth_trajs.loc[(smooth_trajs.particle == 43)]
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(df1.x, df1.y, label = 'Original', zorder = 20)
ax.plot(df2.x, df2.y, label = 'Smoothed', zorder = 10)
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]', title = 'Raw trajectory')
ax.legend()
plt.show()

In [ ]:
def get_trajs(nDrops, red_particle_idx, trajs, subsample_factor, fps):
    blueTrajs = []
    redTrajs = []
    for i in range(0, nDrops):
        if i in red_particle_idx:
            p = trajs.loc[trajs.particle == i, ['x','y']][::subsample_factor]
            redTrajs.append(Trajectory(p.x, p.y, dt = 1/fps*subsample_factor, traj_id=i, diff_est={'method':DiffMethod.LINEAR_DIFF, 
                                                                                  'window_type': WindowType.CENTRAL}))
        else:
            p = trajs.loc[trajs.particle == i, ['x','y']][::subsample_factor]
            blueTrajs.append(Trajectory(p.x, p.y, dt = 1/fps*subsample_factor, traj_id=i))
    return blueTrajs, redTrajs

In [ ]:
if video_selection == '25b25r':
    red_particle_idx = np.sort(np.array([38, 42, 25, 4, 23, 13, 45, 33, 46, 29, 10, 3, 35, 18, 12, 0, 27, 19, 26, 47, 7, 48, 21, 20, 22], dtype=int))
elif video_selection == '49b1r':
    red_particle_idx = np.array([19]).astype(int)
trajs_yupi_b, trajs_yupi_r = get_trajs(nDrops, red_particle_idx, interp_trajectories_initial_part, 1, fps)
turning_angles_b = ys.turning_angles_ensemble(trajs_yupi_b, centered = True)
turning_angles_r = ys.turning_angles_ensemble(trajs_yupi_r, centered = True)

smooth_trajs_yupi_b, smooth_trajs_yupi_r = get_trajs(nDrops, red_particle_idx, smooth_trajs, 1, fps)
smooth_turning_angles_b = ys.turning_angles_ensemble(smooth_trajs_yupi_b, centered = True)
smooth_turning_angles_r = ys.turning_angles_ensemble(smooth_trajs_yupi_r, centered = True)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
ax[0,0].hist(turning_angles_b, bins = 101, density = True, alpha = 0.5, label = 'Original', color = 'blue')
ax[0,1].hist(smooth_turning_angles_b, bins = 101, density = True, alpha = 0.5, label = 'Smoothed', color = 'blue')
ax[1,0].hist(turning_angles_r, bins = 101, density = True, alpha = 0.5, label = 'Original', color = 'red')
ax[1,1].hist(smooth_turning_angles_r, bins = 101, density = True, alpha = 0.5, label = 'Smoothed', color = 'red')
plt.show()